In [36]:
import pandas as pd
import spacy
train = pd.read_csv('C:/Users/dschw/Documents/GitHub/Thesis/Output/NCCR_combined_corpus_DE_wording_available_TRAIN.csv')

In [88]:
test = [train.text_prep[100], train.text_prep[10]]

In [97]:
nlp = spacy.load("de_core_news_sm")
nlp.pipeline


[('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec at 0x25b90db8e50>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x25b1f784d10>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x25b00b58540>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x25b1f781940>),
 ('ner', <spacy.pipeline.ner.EntityRecognizer at 0x25b1f781e20>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x25b006509c0>),
 ('lemmatizer', <spacy.pipeline.lemmatizer.Lemmatizer at 0x25b31fbb600>)]

In [98]:
for doc in nlp.pipe(test):
    #for chunk in doc.noun_chunks:
        #print(chunk.text, chunk.root.text, chunk.root.dep_, chunk.root.head.text)
    #print([(token.text, token.lemma_, token.pos_, token.tag_, token.dep_) for token in doc])
    for ent in doc.ents:
        print(ent.text, ent.label_)

C. Fuer PER
Anrainer-Rechten ORG
Kyoto-Ziele LOC
Luftschadstoffen LOC
laestiger MISC
SPOe ORG
Lebensmittelkontrolle LOC
KonsumentInnen ORG
erhoehen MISC
KonsumentInnenschutz MISC
Agrarsektor LOC
soziale MISC
Oesterreich LOC
Oesterreich LOC
NA Berlin ORG
SPD ORG
CDU ORG
Klaus-Peter Flosbach PER
SPD ORG
SPD ORG
Union LOC
SPD ORG
Union ORG
FDP ORG
Schlupflöcher MISC
Erbschaftsteuer LOC
Staaten LOC
OECD-Standard ORG
Informationsaustausch MISC
USA LOC
EU-Ebene ORG
Schweiz LOC
deutschen MISC
Schweiz LOC
deutschen MISC
Deutschland LOC
Bundesfinanzminister Schäuble PER
SPD ORG
OECD ORG
Niedrigsteuerländer LOC
SPD ORG
Peer Steinbrück PER
Kampagne LOC
SPD ORG
Grüne ORG
Linke ORG
Jahressteuergesetz 2009 MISC
Verfolgungsverjährungsfrist für Steuerhinterziehung ORG
Die jetzige Koalition MISC
CDU/CSU ORG
FDP ORG
Selbstanzeige MISC
Regierungsprogramm LOC
CDU ORG
Deutschlands LOC
OECD-Standard ORG
OECD-Standard ORG
Deutschland LOC
Staaten LOC
SPD ORG
europäischen MISC
Deutschland LOC
EU- ORG
europäisc

In [9]:
train.text_prep

0        3. Vollbeschaeftigung: Sichere Arbeitsplaetze ...
1        In jedem andern Land wäre ein Botschafter wie ...
2        Gastbeitrag von Christian Lindner zur Energiep...
3        Unternehmer und Arbeitgeber fordern mehr Flexi...
4        NA Berlin (ots) - Der Bundesrat hat mit der Me...
                               ...                        
10084    NA Die Alternative für Deutschland hat heute h...
10085    Am 24. November 2013 stimmt die Schweizer Bevö...
10086    13. Service Public: Weiterentwicklung des Serv...
10087    Sicherheit in unserer Gesellschaft Sicherheit ...
10088    Frau Merkel sagt erst, wir schaffen das, und w...
Name: text_prep, Length: 10089, dtype: object

In [ ]:
 any([ent.label_ == "PERSON" for ent in x.doc.ents]):

## Using text2vec

In [25]:
words = ['Volk', 'wir', 'unser', 'Menschen', 'Buerger']

In [26]:
# compute similarity    
similarities = {}   
for word in words:
    tok = nlp(word)
    similarities[tok.text] ={}
    for tok_ in doc:
        similarities[tok.text].update({tok_.text:tok.similarity(tok_)})

# sort
top3 = lambda x: {k: v for k, v in sorted(similarities[x].items(), key=lambda item: item[1], reverse=True)[:3]}

# desired output
for word in words:
    print(word + ': ' + str(top3(word)))

Volk: {'Land': 0.6207446046839928, 'Wahrheit': 0.518788093218624, 'Bürger': 0.48461225568860855}
wir: {'zu': 0.4585251315739777, 'würden': 0.4335937646412793, 'sie': 0.416518505810148}
unser: {'Das': 0.3174341939140802, 'dem': 0.28733673489769995, 'einem': 0.2684855539955589}
Menschen: {'sondern': 0.5892384551544818, 'Erwachen': 0.588274366768848, 'Bürger': 0.5604004880189845}
Buerger: {'Bürger': 0.5391648056956431, 'Schneider': 0.39189663842019057, 'Sozialdemokraten': 0.35841752216105116}


<ipython-input-26-319980e2580b>:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  similarities[tok.text].update({tok_.text:tok.similarity(tok_)})


## Using Transformers

In [129]:
import spacy
import torch
import numpy
from numpy.testing import assert_almost_equal

In [130]:
nlp_trf = spacy.load("de_dep_news_trf")
example_doc = nlp_trf("Helsinki ist die Hauptstadt")
# Check the length of the Doc object
example_doc.__len__()
example_doc[0], example_doc._.trf_data.align[0].data

(Helsinki, array([[1]], dtype=int32))

In [131]:
for doc in nlp.pipe([text]):
    print(doc._.trf_data.tokens['input_texts'])
    print(doc.__len__())
    tokvecs = doc._.trf_data.tensors[0]
    print(tokvecs)

[['[CLS]', 'NA', 'Berlin', '(', 'o', '##ts', ')', '-', 'Der', 'Bundesrat', 'hat', 'mit', 'der', 'Mehrheit', 'der', 'rot', '-', 'rot', '-', 'grün', 'regiert', '##en', 'Länder', 'am', 'heutigen', 'Freitag', 'das', 'Gesetz', 'zur', 'Bekämpfung', 'des', 'Menschen', '##handels', 'und', 'Überwachung', 'von', 'Prostitu', '##ti', '##ons', '##stätten', 'an', 'den', 'Vermittlungs', '##ausschuss', 'überwie', '##sen', '.', 'Damit', 'kann', 'es', 'in', 'dieser', 'Legislatur', '##periode', 'nicht', 'mehr', 'in', 'Kraft', 'gesetzt', 'werden', '.', 'Dazu', 'erklärt', 'der', 'Vorsitzende', 'der', 'AG', 'Innen', 'der', 'CDU', '/', 'CSU', '-', 'Bundestags', '##fraktion', ',', 'Hans', '-', 'Peter', 'U', '##hl', ':', '"', 'Rot', '-', 'Rot', '-', 'Grün', 'lässt', 'das', 'Gesetz', 'zur', 'Bekämpfung', 'von', 'Menschen', '##handel', 'und', 'Zwangs', '##pro', '##stitution', 'scheitern', '.', 'Das', 'ist', 'z', '##yn', '##isch', '.', 'Erst', 'durch', 'das', 'Prostitu', '##ti', '##ons', '##gesetz', ',', 'das', '

In [133]:
# Import the Language object under the 'language' module in spaCy,
# and NumPy for calculating cosine similarity.
from spacy.language import Language
import numpy as np

# We use the @ character to register the following Class definition
# with spaCy under the name 'tensor2attr'.
@Language.factory('tensor2attr')

# We begin by declaring the class name: Tensor2Attr. The name is 
# declared using 'class', followed by the name and a colon.
class Tensor2Attr:
    
    # We continue by defining the first method of the class, 
    # __init__(), which is called when this class is used for 
    # creating a Python object. Custom components in spaCy 
    # require passing two variables to the __init__() method:
    # 'name' and 'nlp'. The variable 'self' refers to any
    # object created using this class!
    def __init__(self, name, nlp):
        
        # We do not really do anything with this class, so we
        # simply move on using 'pass' when the object is created.
        pass

    # The __call__() method is called whenever some other object
    # is passed to an object representing this class. Since we know
    # that the class is a part of the spaCy pipeline, we already know
    # that it will receive Doc objects from the preceding layers.
    # We use the variable 'doc' to refer to any object received.
    def __call__(self, doc):
        
        # When an object is received, the class will instantly pass
        # the object forward to the 'add_attributes' method. The
        # reference to self informs Python that the method belongs
        # to this class.
        self.add_attributes(doc)
        
        # After the 'add_attributes' method finishes, the __call__
        # method returns the object.
        return doc
    
    # Next, we define the 'add_attributes' method that will modify
    # the incoming Doc object by calling a series of methods.
    def add_attributes(self, doc):
        
        # spaCy Doc objects have an attribute named 'user_hooks',
        # which allows customising the default attributes of a 
        # Doc object, such as 'vector'. We use the 'user_hooks'
        # attribute to replace the attribute 'vector' with the 
        # Transformer output, which is retrieved using the 
        # 'doc_tensor' method defined below.
        doc.user_hooks['vector'] = self.doc_tensor
        
        # We then perform the same for both Spans and Tokens that
        # are contained within the Doc object.
        doc.user_span_hooks['vector'] = self.span_tensor
        doc.user_token_hooks['vector'] = self.token_tensor
        
        # We also replace the 'similarity' method, because the 
        # default 'similarity' method looks at the default 'vector'
        # attribute, which is empty! We must first replace the
        # vectors using the 'user_hooks' attribute.
        doc.user_hooks['similarity'] = self.get_similarity
        doc.user_span_hooks['similarity'] = self.get_similarity
        doc.user_token_hooks['similarity'] = self.get_similarity
    
    # Define a method that takes a Doc object as input and returns 
    # Transformer output for the entire Doc.
    def doc_tensor(self, doc):
        
        # Return Transformer output for the entire Doc. As noted
        # above, this is the last item under the attribute 'tensor'.
        # Average the output along axis 0 to handle batched outputs.
        return doc._.trf_data.tensors[-1].mean(axis=0)
    
    # Define a method that takes a Span as input and returns the Transformer 
    # output.
    def span_tensor(self, span):
        
        # Get alignment information for Span. This is achieved by using
        # the 'doc' attribute of Span that refers to the Doc that contains
        # this Span. We then use the 'start' and 'end' attributes of a Span
        # to retrieve the alignment information. Finally, we flatten the
        # resulting array to use it for indexing.
        tensor_ix = span.doc._.trf_data.align[span.start: span.end].data.flatten()
        
        # Fetch Transformer output shape from the final dimension of the output.
        # We do this here to maintain compatibility with different Transformers,
        # which may output tensors of different shape.
        out_dim = span.doc._.trf_data.tensors[0].shape[-1]
        
        # Get Token tensors under tensors[0]. Reshape batched outputs so that
        # each "row" in the matrix corresponds to a single token. This is needed
        # for matching alignment information under 'tensor_ix' to the Transformer
        # output.
        tensor = span.doc._.trf_data.tensors[0].reshape(-1, out_dim)[tensor_ix]
        
        # Average vectors along axis 0 ("columns"). This yields a 768-dimensional
        # vector for each spaCy Span.
        return tensor.mean(axis=0)
    
    # Define a function that takes a Token as input and returns the Transformer
    # output.
    def token_tensor(self, token):
        
        # Get alignment information for Token; flatten array for indexing.
        # Again, we use the 'doc' attribute of a Token to get the parent Doc,
        # which contains the Transformer output.
        tensor_ix = token.doc._.trf_data.align[token.i].data.flatten()
        
        # Fetch Transformer output shape from the final dimension of the output.
        # We do this here to maintain compatibility with different Transformers,
        # which may output tensors of different shape.
        out_dim = token.doc._.trf_data.tensors[0].shape[-1]
        
        # Get Token tensors under tensors[0]. Reshape batched outputs so that
        # each "row" in the matrix corresponds to a single token. This is needed
        # for matching alignment information under 'tensor_ix' to the Transformer
        # output.
        tensor = token.doc._.trf_data.tensors[0].reshape(-1, out_dim)[tensor_ix]

        # Average vectors along axis 0 (columns). This yields a 768-dimensional
        # vector for each spaCy Token.
        return tensor.mean(axis=0)
    
    # Define a function for calculating cosine similarity between vectors
    def get_similarity(self, doc1, doc2):
        
        # Calculate and return cosine similarity
        print(doc1.vector, doc2.vector)
        return np.dot(doc1.vector, doc2.vector) / (doc1.vector_norm * doc2.vector_norm)

TypeError: <class '__main__.Tensor2Attr'> is a built-in class

In [134]:
nlp_trf.add_pipe('tensor2attr')

# Call the 'pipeline' attribute to examine the pipeline
nlp_trf.pipeline

[('transformer',
  <spacy_transformers.pipeline_component.Transformer at 0x25b05f061d0>),
 ('tagger', <spacy.pipeline.tagger.Tagger at 0x25b32736310>),
 ('morphologizer',
  <spacy.pipeline.morphologizer.Morphologizer at 0x25b326db130>),
 ('parser', <spacy.pipeline.dep_parser.DependencyParser at 0x25b32306ac0>),
 ('attribute_ruler',
  <spacy.pipeline.attributeruler.AttributeRuler at 0x25b03f90080>),
 ('lemmatizer', <spacy.pipeline.lemmatizer.Lemmatizer at 0x25b1f68f300>),
 ('tensor2attr', <__main__.Tensor2Attr at 0x25b6f53f0d0>)]

In [135]:
key_word = nlp_trf("Helsinki")
comparison_sentence = nlp_trf("Helsinki ist die Hauptstadt Finnlands")
comparison_sentence = nlp_trf("Der Schauspieler heißt im Film Helsinki")

In [136]:
key_word[0]

Helsinki

In [157]:
key_word = nlp_trf("Regierung")
docs = [train.text_prep[100], train.text_prep[10]]

In [159]:
# compute similarity   
similarities = {} 

for doc in nlp_trf.pipe(docs):
    #print(doc)
    for index in range(len(doc)):
        print(doc[index].similarity(key_word[0]))
     

0.20730342
0.10754468
0.17986044
0.1980016
0.5510587
-0.013063358
0.10473642
0.3475478
0.36473265
0.17676131
0.22357307
0.1127155
0.36169863
0.2016577
0.16316028
0.27012688
0.20829844
0.23605715
0.2601968
0.019064669
0.10209194
0.10921202
0.18542391
0.48181412
0.050037123
0.17148829
0.19455488
0.19748424
0.28710625
0.031577066
0.042759687
0.10508172
0.11966971
0.10096864
0.031025216
0.15865353
0.17381646
0.13019764
0.54353064
0.050617278
0.1056992
0.16704547
-0.01853795
0.17718017
0.16046351
0.5370505
0.13467075
0.20194992
0.03566101
0.31488335
0.08726545
0.102053754
0.06648662
0.020499256
0.3914773
0.22299564
0.36996114
0.1314299
0.10390936
-0.001531479
0.19646944
0.033165224
0.18403257
0.096269324
0.08686891
0.42746004
0.0653656
0.12441116
0.54024345
0.005379984
0.106701545
0.09032813
0.16373123
-0.05072521
0.32907185
0.054798648
0.19824196
0.17329212
0.15543555
-0.054136515
0.10340601
0.069504306
0.25988063
0.2286765
0.2504125
0.5248468
0.19067658
0.032793853
0.11303715
0.12919986
0

0.09610802
0.07067798
0.108725674
0.21920183
0.2516584
0.06297579
0.12596472
0.20520303
0.10346998
0.5770256
0.19449961
0.10255299
0.22013345
0.16011189
0.06530402
0.03594288
0.19435038
0.044555087
0.21220699
0.11237215
0.04776752
0.30080926
0.067708425
0.020562917
0.068524815
0.066603824
0.18011606
0.17696047
0.09659246
0.03254881
0.061512575
0.098207
0.24157806
0.24347942
0.6600304
0.15760088
0.24157397
0.02407094
0.24546039
0.047522362
-0.064408004
0.20358935
0.13038464
0.040979918
0.052768897
0.2245826
0.24733855
0.3064932
0.17273997
0.06631975
0.16419809
0.18004788
0.12800768
0.13922909
0.31240982
0.17466174
0.21621099
0.2050097
0.18765458
0.25847465
0.18512475
0.14086133
0.2321299
0.12621973
0.10618646
0.18776317
0.07179811
0.22933793
0.09685143
0.07250437
0.045634206
0.06529916
0.16089162
0.2679599
0.037612688
0.18641718
0.27177846
0.1394807
0.21618797
0.14696527
0.2256548
0.10320581
0.016996464
-0.0062486986
0.08601015
0.165303
0.11154402
0.040664554
0.09507977
0.15223785
0.093

In [138]:
comparison_sentence[0].similarity(key_word[0])

0.35726312

In [153]:
# compute similarity    
similarities = {}   
for word in words:
    tok = nlp(word)
    similarities[tok.text] ={}
    for tok_ in doc:
        similarities[tok.text].update({tok_.text:tok.similarity(tok_)})

# sort
top3 = lambda x: {k: v for k, v in sorted(similarities[x].items(), key=lambda item: item[1], reverse=True)[:3]}

# desired output
for word in words:
    print(word + ': ' + str(top3(word)))

In [140]:
for sentence in sentences:
for index in range(len(comparison_sentence)):
    print(index)
    print(comparison_sentence[index].similarity(key_word[0]))

0
0.35726312
1
0.27027044
2
0.38636944
3
0.14431223
4
0.115779296
5
0.875999


In [160]:
print(comparison_sentence[0].similarity(key_word[0]))

0.36930782


In [81]:
trf_1 = nlp_trf("Wir stehen politisch zu den Menschen")
trf_2 = nlp_trf("Unsere Partei steht auf der Seite der Bevölkerung")
trf_1[0].similarity(trf_2[1])

0.3584354